# Goal

As I have shown in the previous work the weight of vote varies by constituency (electoral district), apart from widely known issue of 41 independents races with D’Hondt approbation method (the rounding). 

Here I would like to focus on the first, to create a wrapper for polls i.e weights for each party voters base, the explanation for possible situation when party such as KO gets first place and more votes over party like PiS (and thus getting D’Hondt rounding boost), however despite getting more votes overall they would gain fewer seats as the weight of their votes is less, ie the regions with weaker votes (such as Warszawa and Poznań) being overrepresented in the results, while regions with stronger votes (Chełm, Nowy Sącz, Elbląg) being underrepresented.

(In this chapter we will just load the data without the Minio framework that was required in my university course)

In [1]:
import pandas as pd
import numpy as np
from io import BytesIO
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import json

In [2]:
# Prepare data
df = pd.read_csv('wykaz_list_sejm_2023.csv', sep=';')
df = df[df['Udział w podziale mandatów'] == 'Tak']
df = df.drop(df.columns[[1,2, 3, 5, 6]], axis=1)

In [3]:
# c = df.iloc[:, [2, 5]]
# Get constituency data
cols = df.columns
print(cols)
df[cols[2]] = df[cols[2]].astype('string').str.replace(',', '.').astype('float32')

Index(['Numer okręgu', 'Liczba głosów', 'Liczba mandatów'], dtype='object')


In [9]:

df = df.groupby(cols[0]).sum()
df['Wskaznik'] = (df[cols[1]] / df[cols[2]]).astype(int)
df['Waga'] = df['Wskaznik'].mean() / df['Wskaznik']
dfw = df.sort_values('Waga')
dfw

,Liczba głosów,Liczba mandatów,Waga,Wskaznik
Numer okręgu,,,,
19,1668661,20.0,0.531204,83433
39,586561,10.0,0.755592,58656
20,702590,12.0,0.756972,58549
3,742457,14.0,0.835721,53032
13,734990,14.0,0.844206,52499
25,599299,12.0,0.887447,49941
27,425471,9.0,0.937513,47274
26,657501,14.0,0.943701,46964
41,536933,12.0,0.990523,44744


In [14]:
df = pd.read_csv('wykaz_list_sejm_2023.csv', sep=';')
df = df[df['Udział w podziale mandatów'] == 'Tak']
df.head()
# Data

,Numer okręgu,Nr listy,Nazwa Komitetu,Liczba kandydatów,Liczba głosów,Procent głosów oddanych w okręgu,Udział w podziale mandatów,Liczba mandatów
1,1,2,KOALICYJNY KOMITET WYBORCZY TRZECIA DROGA POLS...,24,53958,"10,75",Tak,1
2,1,3,KOMITET WYBORCZY NOWA LEWICA,24,47715,"9,51",Tak,1
3,1,4,KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ,24,174643,"34,80",Tak,5
4,1,5,KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPOD...,24,31770,"6,33",Tak,0
5,1,6,KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELS...,24,169540,"33,78",Tak,5


# Results 
How the geographical location of each party's electorate boosted their final result:

In [141]:
cols = df.columns
# Raw results
lb = df.groupby(cols[2]).sum()[cols[4]]
counter = {}

for committy in df[cols[2]].unique():
    sum_weighted = 0
    for cons_no in df[cols[0]].unique():
        row = df[(df[cols[0]] == cons_no) & (df[cols[2]] == committy)]
        
        if not row.empty:
            votes = row[cols[4]].iloc[0]
            weight = dfw.loc[cons_no]['Waga']
            val = weight * votes
            sum_weighted += val

    counter[committy] = round(sum_weighted)

# Lewica party loses most, set as base relative base point
base_factor = lb['KOMITET WYBORCZY NOWA LEWICA'] / counter['KOMITET WYBORCZY NOWA LEWICA']

print("Party - Real result - Official result - Location boost (relative)")
for k in counter:
    scale = base_factor * counter[k] / lb[k] - 1
    scale = round(100*scale, 4)
    print(f"{" ".join(k.split(" ")[-3:])}: {counter[k]} {lb[k]} {scale}%")

print("\n\nBoost relative to KO")
ko_name = 'KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI'
base_factor = lb[ko_name] / counter[ko_name]
for k in counter:
    scale = base_factor * counter[k] / lb[k] - 1
    scale = round(100*scale, 4)
    print(f"{k}: {scale}%")



Party - Real result - Official result - Location boost (relative)
POLSKIE STRONNICTWO LUDOWE: 3046380 3110670 3.2971%
WYBORCZY NOWA LEWICA: 1762486 1859018 -0.0%
PRAWO I SPRAWIEDLIWOŚĆ: 7719589 7640854 6.5639%
WOLNOŚĆ I NIEPODLEGŁOŚĆ: 1518394 1547364 3.5023%
.N IPL ZIELONI: 6311015 6629402 0.4113%
WYBORCÓW MNIEJSZOŚĆ NIEMIECKA: 29541 25778 20.8743%


Boost relative to KO
KOALICYJNY KOMITET WYBORCZY TRZECIA DROGA POLSKA 2050 SZYMONA HOŁOWNI - POLSKIE STRONNICTWO LUDOWE: 2.8739%
KOMITET WYBORCZY NOWA LEWICA: -0.4097%
KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ: 6.1274%
KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPODLEGŁOŚĆ: 3.0783%
KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI: 0.0%
KOMITET WYBORCZY WYBORCÓW MNIEJSZOŚĆ NIEMIECKA: 20.3791%
